# 取得 LiDAR 資料

包含安裝 Matplotlib 的指令。執行一次後即可跳過此步驟。首次安裝後需要重新啟動 kernel。

In [ ]:
import math

import kachaka_api
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from IPython.display import Image, clear_output

client = kachaka_api.aio.KachakaApiClient()
await client.set_manual_control_enabled(True)


async def get_and_show_laser_scan_loop():
    async for scan in client.ros_laser_scan.stream():
        clear_output(wait=True)
        fig = plt.figure(figsize=(5, 5))

        n = len(scan.ranges)
        x = list(range(n))
        y = list(range(n))
        for i in range(n):
            theta = scan.angle_min + scan.angle_increment * i
            x[i] = scan.ranges[i] * math.cos(theta)
            y[i] = scan.ranges[i] * math.sin(theta)

        plt.plot(0, 0, "o", color="black")
        plt.plot(x, y, ".")
        plt.xlim(-6.0, 6.0)
        plt.ylim(-6.0, 6.0)
        plt.grid(True)
        plt.gca().set_aspect("equal", adjustable="box")
        plt.show()


await get_and_show_laser_scan_loop()

上述方法是透過開啟手動操控模式來驅動 LiDAR，當 Kachaka 停止移動一段時間後，掃描將會停止。
如下所示，使用 Activator 可以在 Kachaka 靜止的狀態下繼續進行掃描。

但請注意，**持續不必要地運轉掃描將會大幅縮短 LiDAR 的產品壽命**。

In [ ]:
from kachaka_api.util.vision import LaserScanActivator

activator = LaserScanActivator()
with activator.activate():
    await get_and_show_laser_scan_loop()